# 3. Run ML Models
This notebook executes both Logistic Regression and XGBoost models on the full set of features as well as a reduced set of top two features, aiming to compare the performance across these scenarios. The results are then visualized, specifically focusing on the ROC curves, and relevant performance metrics are stored for future reference and analysis.

### Objective:
To execute various machine learning models using the dataset provided and assess their performance on predicting the given outcome.

### Data Overview:

* Source: The data for this notebook is sourced from various CSV files located within directories defined in the notebook.
* Features: The dataset contains a mix of numerical and categorical features. Some key features include 'start_glc', 'duration', and many others.
* Target Variable: The prediction target is 'y_3', which is possibly a binary outcome indicating a certain event or condition.

### Sections:

1. Setup: Importing necessary libraries and defining paths.
2. Data Loading: Reading the required datasets from their respective directories.
3. Data Preparation: Setting up dataframes to store results and setting up predictor variables and target variable.
4. Model Execution:
        All Features:
            Logistic Regression: Execution of logistic regression using all features, hyperparameter tuning, and storing of results.
            XGBoost: Execution of XGBoost using all features and storing of results.
        Top Two Features:
            Logistic Regression: Execution of logistic regression using only the top two features, 'start_glc' and 'duration', and storing of results.
            XGBoost: Execution of XGBoost using only the top two features and storing of results.
5. Results Compilation: Storing of model results, calculation of mean results, and appending of results to dataframes.
6. Data Saving: Storing results in specified directories.

## 3.0. Packages and data

In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)
import random
random.seed(42)
from sklearn.metrics import roc_curve, auc
import ml_helper as ml_help
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pickle
import sys
path = "../../diametrics"
sys.path.append(path)

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
# Directories
directory = '../../data/tidy_data/final_df/'
probability_results_directory = '../../results/probability_results/'
k_fold_results_directory = '../../results/k_fold_results/'
threshold_results_directory = '../../results/threshold_results/'
mean_results_directory = '../../results/mean_results/'
dict_directory = '../../results/dict_results/'

In [3]:
# Read data
df = pd.read_csv(directory + 'df.csv')
strat = df['stratify']
X = pd.read_csv(directory + 'X.csv')
y = df['y']

In [6]:
# Set up dfs for results
df_with_probas = df.copy()
mean_results = pd.DataFrame()

## 3.1. All features

### 3.1.1. Logistic regression

# Run the LR model, uses hyperopt for HP tuning, get accuracy, indices and probabilities for each fold
lr_all_k_fold_results, lr_all_test_sets_index, lr_all_predicted_probas, lr_all_observed, lr_all_shap_values, lr_all_coeffs, lr_all_hps = ml_help.k_fold_accuracies(X, y, strat, True, 'all')

# Store results in dictionary
lr_all_results = {'X':X,
              'probas':lr_all_predicted_probas, 
              'observed':lr_all_observed, 
              'shap':lr_all_shap_values, 
              'coeffs':lr_all_coeffs}

with open(dict_directory+"lr_results_all", "wb") as fp:   
    #Pickling 
    pickle.dump(lr_all_results, fp) 

# Add the mean accuracy to a table for easy perusal
mean_results = ml_help.add_mean_to_df(mean_results, lr_all_k_fold_results, 'lr', 'all')

In [ ]:
# Add a probability column to the whole dataset to ensure 
df_with_probas = ml_help.add_proba_col(df_with_probas, lr_all_test_sets_index, lr_all_predicted_probas, 'probas_lr_all')

In [ ]:
# Save k-fold results
lr_all_k_fold_results.to_csv(k_fold_results_directory+'lr_all.csv', index=False)

In [ ]:
# Save hyperparameters
pd.DataFrame(lr_all_hps).to_csv('../../results/hyperparameters/lr_all.csv', index=False)

### 3.1.2. XGBoost

In [7]:
# Run the XGBoost model, uses optuna for HP tuning, get accuracy, indices and probabilities for each fold
xgb_ts_k_fold_results, xgb_ts_test_sets_index, xgb_ts_predicted_probas, xgb_ts_observed, xgb_ts_shap, _, xgb_ts_hps = ml_help.k_fold_accuracies(X, y, strat, False, 'all')

[I 2023-10-30 11:42:08,095] A new study created in memory with name: no-name-cc79c65e-7b65-4e5e-bb54-6257b36abff7
[I 2023-10-30 11:43:31,185] Trial 0 finished with value: 0.8805406 and parameters: {'n_estimators': 360, 'max_depth': 10, 'min_child_weight': 2, 'subsample': 0.88069840519828, 'colsample_bytree': 0.7745339339541701, 'eta': 0.12663788179159863, 'learning_rate': 0.2709552449490183, 'reg_alpha': 2, 'reg_lambda': 5, 'gamma': 1}. Best is trial 0 with value: 0.8805406.
[I 2023-10-30 11:45:01,949] Trial 1 finished with value: 0.8834674000000001 and parameters: {'n_estimators': 870, 'max_depth': 9, 'min_child_weight': 5, 'subsample': 0.8117659031059572, 'colsample_bytree': 0.9785914398424478, 'eta': 0.22353632863973977, 'learning_rate': 0.030517321256285642, 'reg_alpha': 0, 'reg_lambda': 1, 'gamma': 0}. Best is trial 1 with value: 0.8834674000000001.
[I 2023-10-30 11:45:39,009] Trial 2 finished with value: 0.8943000999999999 and parameters: {'n_estimators': 799, 'max_depth': 4, 'mi

In [8]:
# Store results in dictionary
xgb_ts_results = {'X':X,
                'probas':xgb_ts_predicted_probas, 
              'observed':xgb_ts_observed, 
              'shap':xgb_ts_shap
              }

with open(dict_directory+"xgb_ts", "wb") as fp:   
    #Pickling 
    pickle.dump(xgb_ts_results, fp)

In [9]:
# Add the mean accuracy to a table for easy perusal
mean_results = ml_help.add_mean_to_df(mean_results, xgb_ts_k_fold_results, 'xgb', 'all')

In [10]:
# Add a probability column to the whole dataset to ensure 
df_with_probas = ml_help.add_proba_col(df_with_probas, xgb_ts_test_sets_index, xgb_ts_predicted_probas , 'probas_xgb_ts')

In [11]:
# Save k-fold results
xgb_ts_k_fold_results.to_csv(k_fold_results_directory+'xgb_ts.csv', index=False)

In [12]:
# Save hyperparameters
pd.DataFrame(xgb_ts_hps).to_csv('../../results/hyperparameters/xgb_ts.csv', index=False)

## 3.2. Two features
Two features shown in the feature selection process to be the most important, start glucose and duration of exercise bout

In [13]:
# Select the two features from feature selection
X_two = X[['start_glc','duration']]

### 3.2.1. Logistic regression

In [ ]:
# Run the LR model, uses hyperopt for HP tuning, get accuracy, indices and probabilities for each fold
lr_two_k_fold_results, lr_two_test_sets_index, lr_two_predicted_probas, lr_two_observed, lr_two_shap_values, lr_two_coeffs, lr_two_hps = ml_help.k_fold_accuracies(X_two, y, strat, True, 'two')

In [ ]:
# Store results in dictionary
lr_two_results = {'X':X_two,
              'probas':lr_two_predicted_probas, 
              'observed':lr_two_observed, 
              'shap':lr_two_shap_values, 
              'coeffs':lr_two_coeffs}

with open(dict_directory+"lr_two", "wb") as fp:   
    #Pickling 
    pickle.dump(lr_two_results, fp) 

In [ ]:
# Add the mean accuracy to a table for easy perusal
mean_results = ml_help.add_mean_to_df(mean_results, lr_two_k_fold_results, 'lr', 'two')

In [ ]:
# Add a probability column to the whole dataset to ensure 
df_with_probas = ml_help.add_proba_col(df_with_probas, lr_two_test_sets_index, lr_two_predicted_probas, 'probas_lr_two')

In [ ]:
# Save k-fold results
lr_two_k_fold_results.to_csv(k_fold_results_directory+'lr_two.csv', index=False)

In [ ]:
# Save hyperparameters
pd.DataFrame(lr_two_hps).to_csv('../../results/hyperparameters/lr_two.csv', index=False)

### 3.2.2. XGB Two feat

In [14]:
# Run the XGBoost model, uses optuna for HP tuning, get accuracy, indices and probabilities for each fold
xgb_two_k_fold_results, xgb_two_test_sets_index, xgb_two_predicted_probas, xgb_two_observed, xgb_two_shap, _, xgb_two_hps = ml_help.k_fold_accuracies(X_two, y, strat, False, 'two')

[I 2023-10-30 14:13:19,690] A new study created in memory with name: no-name-a5540dc0-2192-4ddf-a9ab-124bfd1b0930
[I 2023-10-30 14:13:24,618] Trial 0 finished with value: 0.8405521 and parameters: {'n_estimators': 207, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.6788315281094192, 'colsample_bytree': 0.2565112644833512, 'eta': 0.13154834097839968, 'learning_rate': 0.3787177706320947, 'reg_alpha': 2, 'reg_lambda': 4, 'gamma': 1}. Best is trial 0 with value: 0.8405521.
[I 2023-10-30 14:13:28,523] Trial 1 finished with value: 0.8425812 and parameters: {'n_estimators': 736, 'max_depth': 11, 'min_child_weight': 2, 'subsample': 0.9097718966999266, 'colsample_bytree': 0.22107554840927018, 'eta': 0.2853302193665385, 'learning_rate': 0.018449659902107726, 'reg_alpha': 4, 'reg_lambda': 2, 'gamma': 2}. Best is trial 1 with value: 0.8425812.
[I 2023-10-30 14:13:34,891] Trial 2 finished with value: 0.8461703999999999 and parameters: {'n_estimators': 21, 'max_depth': 11, 'min_child_weight':

In [15]:
# Store results in dictionary
xgb_two_results = {'X':X_two, 
                'probas':xgb_two_predicted_probas, 
                'observed':xgb_two_observed, 
                'shap':xgb_two_shap
                }

with open(dict_directory+"xgb_results_two", "wb") as fp:   
    #Pickling 
    pickle.dump(xgb_two_results, fp)

In [16]:
# Add the mean accuracy to a table for easy perusal
mean_results = ml_help.add_mean_to_df(mean_results, xgb_two_k_fold_results, 'xgb', 'two')

In [17]:
mean_results

,roc_auc,mae,logloss,brier,threshold,accuracy,precision,recall,f1,predicted_positive_rate,observed_positive_rate,tpr,fpr,specificity,balanced_accuracy,model,features
mean,0.889252,0.107016,0.210281,0.059696,0.060,0.782025,0.286857,0.830823,0.422824,0.280530,0.0944,0.830823,0.223097,0.776903,0.803863,xgb,all
mean,0.848677,0.126934,0.227582,0.064119,0.082,0.750451,0.251883,0.780359,0.377385,0.302469,0.0944,0.780359,0.252574,0.747426,0.763893,xgb,two


In [18]:
# Add a probability column to the whole dataset to ensure 
df_with_probas = ml_help.add_proba_col(df_with_probas, xgb_two_test_sets_index, xgb_two_predicted_probas , 'probas_xgb_two')

In [19]:
# Save k-fold results
xgb_two_k_fold_results.to_csv(k_fold_results_directory+'xgb_two.csv', index=False)

In [20]:
# Save hyperparameters
pd.DataFrame(xgb_two_hps).to_csv('../../results/hyperparameters/xgb_two.csv', index=False)

In [21]:
# Save dataframe with all predicted probas
df_with_probas.to_csv('../../results/probability_results/xgb_two.csv')